# Fast SLAM

## 8.3 implement paricle filter

In [ ]:
import sys
sys.path.append('../scripts/')
from kf import * # this also import Mcl class
%matplotlib widget

In [ ]:
class FastSlam(Mcl):
    def __init__(self, envmap, init_pose, num_particle, num_landmark, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2},                  
                 distance_dev_rate=0.14, direction_dev=0.05):
        super().__init__(envmap, init_pose, num_particle, motion_noise_stds, distance_dev_rate, direction_dev)

In [ ]:
def trial(cambias=True):
    time_interval = 0.1
    world = World(30, time_interval, debug = False)
    
    m = Map() # defined in ideal_robot.py
    for landmark in [(-4,2),(2,-3),(3,3)]: 
        m.append_landmark((Landmark(*landmark)))
    world.append(m)
    
    init_pose = np.array([0,0,0]).T
    num_particle = 100
    fastslam = FastSlam(m, init_pose, num_particle, len(m.landmarks))
    nu = 0.2
    omega = 10.0/180*math.pi
    agent = EstimationAgent(time_interval, nu, omega, fastslam) # in Mcl class
    
    if cambias:
        robot = Robot(init_pose, sensor=Camera(m), agent=agent, color='red')
    else:
        robot = Robot(init_pose, sensor=Camera(m,distance_bias_rate_stddev=0,direction_bias_stddev=0), agent=agent, color='red')
    
    world.append(robot)
    
    world.draw()

In [ ]:
trial(cambias=False)

In [ ]:
class EstimatedLandmark(Landmark):
    def __init__(self):
        super().__init__(0,0)
        self.cov = np.array([[1,0],[0,2]])
        
    def draw(self, ax, elems):
        if self.cov is None:
            return
        
        center = ax.scatter(self.pos[0], self.pos[1], s=100, marker="*", label="landmarks", color="blue")
        elems.append(center)
        elems.append(ax.text(self.pos[0], self.pos[1], "id:"+str(self.id), fontsize=10))
        
        ellipse = sigma_ellipse(self.pos, self.cov, 3)
        elems.append(ax.add_patch(ellipse))

In [ ]:
class MapParticle(Particle):
    def __init__(self, init_pose, weight, landmark_num):
        super().__init__(init_pose, weight)
        
        self.map = Map()
        for i in range(landmark_num):
            self.map.append_landmark(EstimatedLandmark())

In [ ]:
class FastSlam(Mcl):
    def __init__(self, envmap, init_pose, num_particle, num_landmark, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2},                  
                 distance_dev_rate=0.14, direction_dev=0.05):
        super().__init__(envmap, init_pose, num_particle, motion_noise_stds, distance_dev_rate, direction_dev)
        self.particles = [MapParticle(init_pose, 1.0/num_particle, num_landmark) for i in range(num_particle)] # redefine Mcl's particles
        self.ml = self.particles[0] # redefine Mcl's ml(maximum likelihood). 
        
    def draw(self, ax, elems):
        super().draw(ax, elems)
        self.ml.map.draw(ax, elems)

In [ ]:
trial()

# 8.4 implement landmark position estimation

In [ ]:
class EstimatedLandmark(Landmark): # add cov to Landmark.
    def __init__(self):
        super().__init__(0,0)
        self.cov = None
        
    def draw(self, ax, elems):
        if self.cov is None:
            return
        
        center = ax.scatter(self.pos[0], self.pos[1], s=100, marker="*", label="landmarks", color="blue")
        elems.append(center)
        elems.append(ax.text(self.pos[0], self.pos[1], "id:"+str(self.id), fontsize=10))
        
        ellipse = sigma_ellipse(self.pos, self.cov, 3)
        elems.append(ax.add_patch(ellipse))

In [ ]:
class MapParticle(Particle):
    def __init__(self, init_pose, weight, landmark_num):
        super().__init__(init_pose, weight)
        
        self.map = Map() # defined in IdealRobot.py
        for i in range(landmark_num):
            self.map.append_landmark(EstimatedLandmark())
            
                
    def init_landmark_estimation(self, landmark, z, distance_dev_rate, direction_dev):
        landmark.pos = (z[0] * np.array([np.cos(self.pose[2]+z[1]), 
                                        np.sin(self.pose[2]+z[1])]).T 
                        + self.pose[0:2]) # z=(x,y,phi), self.pose=(x,y,theta)
        H = -matH(self.pose, landmark.pos)[0:2,0:2] # in kf.py
        Q = matQ(distance_dev_rate*z[0], direction_dev)
        landmark.cov = np.linalg.inv(H.T.dot(np.linalg.inv(Q).dot(H)))
        
    def observation_update(self, observation, distance_dev_rate, direction_dev):
        for d in observation:
            z = d[0]
            landmark = self.map.landmarks[d[1]]
            
            if landmark.cov is None:
                self.init_landmark_estimation(landmark, z, distance_dev_rate, direction_dev)
            else:
                self.observation_update_landmark(landmark, z, distance_dev_rate, direction_dev)
                
    def observation_update_landmark(self, landmark, z, distance_dev_rate, direction_dev):
        est_z = IdealCamera.observation_function(self.pose, landmark.pos) #h(m^)
        if est_z[0] < 0.01:
            return
        
        H = -matH(self.pose, landmark.pos)[0:2,0:2]
        Q = matQ(distance_dev_rate * est_z[0], direction_dev)
        K = landmark.cov.dot(H.T).dot(np.linalg.inv(Q + H.dot(landmark.cov).dot(H.T)))
        landmark.pos += K.dot(z - est_z)
        landmark.cov = (np.eye(2) - K.dot(H)).dot(landmark.cov)

In [ ]:
class FastSlam(Mcl):
    def __init__(self, init_pose, num_particle, num_landmark, motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2},                  
                 distance_dev_rate=0.14, direction_dev=0.05):
        super().__init__(None, init_pose, num_particle, motion_noise_stds, distance_dev_rate, direction_dev) # let envmap None.
        self.particles = [MapParticle(init_pose, 1.0/num_particle, num_landmark) for i in range(num_particle)]
        self.ml = self.particles[0] # temporally
        
    def draw(self, ax, elems):
        super().draw(ax, elems)
        self.ml.map.draw(ax, elems)
        
    def observation_update(self, observation):
        for p in self.particles:
            p.observation_update(observation, self.distance_dev_rate, self.direction_dev) #MapParticle's function
        self.set_ml() # defined in Mcl.
        self.resampling()

In [ ]:
def trial(cambias=True):
    time_interval = 0.1
    world = World(30, time_interval, debug = False)
    
    map = Map() # defined in ideal_robot.py
    for landmark in [(-4,2),(2,-3),(3,3)]: 
        map.append_landmark((Landmark(*landmark)))
    world.append(map)
    
    init_pose = np.array([0,0,0]).T
    num_particle = 100
    fastslam = FastSlam(init_pose, num_particle, len(map.landmarks)) # delete map.
    nu = 0.2 
    omega = 10.0/180*math.pi
    agent = EstimationAgent(time_interval, nu, omega, fastslam) # in Mcl class
    
    if cambias:
        robot = Robot(init_pose, sensor=Camera(map), agent=agent, color='red')
    else:
        robot = Robot(init_pose, sensor=Camera(map,distance_bias_rate_stddev=0,direction_bias_stddev=0), agent=agent, color='red')
    
    world.append(robot)
    
    world.draw()

In [ ]:
trial()

In [ ]:
trial(cambias=False)

# 8.5 update particle's weight

In [ ]:
class MapParticle(Particle):
    def __init__(self, init_pose, weight, landmark_num):
        super().__init__(init_pose, weight)
        
        self.map = Map() # defined in IdealRobot.py
        for i in range(landmark_num):
            self.map.append_landmark(EstimatedLandmark())
            
                
    def init_landmark_estimation(self, landmark, z, distance_dev_rate, direction_dev):
        landmark.pos = (z[0] * np.array([np.cos(self.pose[2]+z[1]), 
                                        np.sin(self.pose[2]+z[1])]).T 
                        + self.pose[0:2]) # z=(x,y,phi), self.pose=(x,y,theta)
        H = -matH(self.pose, landmark.pos)[0:2,0:2] # in kf.py
        Q = matQ(distance_dev_rate*z[0], direction_dev)
        landmark.cov = np.linalg.inv(H.T.dot(np.linalg.inv(Q).dot(H)))
        
    def observation_update(self, observation, distance_dev_rate, direction_dev):
        for d in observation:
            z = d[0]
            landmark = self.map.landmarks[d[1]]
            
            if landmark.cov is None:
                self.init_landmark_estimation(landmark, z, distance_dev_rate, direction_dev)
            else:
                self.observation_update_landmark(landmark, z, distance_dev_rate, direction_dev)
                
    def observation_update_landmark(self, landmark, z, distance_dev_rate, direction_dev):
        est_z = IdealCamera.observation_function(self.pose, landmark.pos) #h(m^)
        if est_z[0] < 0.01:
            return
        
        # landmark update
        H = -matH(self.pose, landmark.pos)[0:2,0:2]
        Q = matQ(distance_dev_rate * est_z[0], direction_dev)
        K = landmark.cov.dot(H.T).dot(np.linalg.inv(Q + H.dot(landmark.cov).dot(H.T)))
        landmark.pos += K.dot(z - est_z)
        landmark.cov = (np.eye(2) - K.dot(H)).dot(landmark.cov)
        
        # trajectory(weight) update
        Q_z = H.dot(landmark.cov).dot(H.T) + Q
        self.weight *= multivariate_normal(mean=est_z, cov=Q_z).pdf(z)


In [ ]:
trial(cambias=False)

In [ ]:
trial()